In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, Embedding, LSTM, Bidirectional
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

import numpy as np

n_unique_words = 10000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=n_unique_words)

maxlen = 200
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
y_train = np.array(y_train)
y_test = np.array(y_test)

def create_model(use_attention=False):
    inputs = Input(shape=(maxlen,))
    x = Embedding(n_unique_words, 128, input_length=maxlen)(inputs)
    x = Bidirectional(LSTM(64, return_sequences=use_attention))(x)
    if use_attention:
        x = attention()(x)
    x = Dropout(0.5)(x)
    x = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

class attention(tf.keras.layers.Layer):
    def __init__(self):
        super(attention, self).__init__()

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1), initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1), initializer="normal")
        super(attention, self).build(input_shape)

    def call(self, x):
        e = K.tanh(K.dot(x, self.W) + self.b)
        a = K.softmax(e, axis=1)
        output = x * a
        return K.sum(output, axis=1)

model1 = create_model(use_attention=False)
model2 = create_model(use_attention=True)

history1 = model1.fit(x_train, y_train, batch_size=256, epochs=12, validation_data=(x_test, y_test))
history2 = model2.fit(x_train, y_train, batch_size=256, epochs=12, validation_data=(x_test, y_test))

print("Model 1:")
print("Loss: ", history1.history['loss'])
print("Accuracy: ", history1.history['accuracy'])

print("Model 2:")
print("Loss: ", history2.history['loss'])
print("Accuracy: ", history2.history['accuracy'])


17464789/17464789 [==============================] - 1s 0us/step
Epoch 1/12
98/98 [==============================] - 28s 197ms/step - loss: 0.5055 - accuracy: 0.7385 - val_loss: 0.3245 - val_accuracy: 0.8641
Epoch 2/12
98/98 [==============================] - 14s 140ms/step - loss: 0.2516 - accuracy: 0.9040 - val_loss: 0.3026 - val_accuracy: 0.8718
Epoch 3/12
98/98 [==============================] - 11s 114ms/step - loss: 0.1802 - accuracy: 0.9360 - val_loss: 0.3329 - val_accuracy: 0.8692
Epoch 4/12
98/98 [==============================] - 9s 95ms/step - loss: 0.1397 - accuracy: 0.9503 - val_loss: 0.3611 - val_accuracy: 0.8616
Epoch 5/12
98/98 [==============================] - 8s 80ms/step - loss: 0.1129 - accuracy: 0.9612 - val_loss: 0.4429 - val_accuracy: 0.8599
Epoch 6/12
98/98 [==============================] - 8s 80ms/step - loss: 0.0885 - accuracy: 0.9708 - val_loss: 0.5124 - val_accuracy: 0.8556
Epoch 7/12
98/98 [==============================] - 8s 85ms/step - loss: 0.0852 - a

In [ ]:
results = model1.evaluate(x_test, y_test)

782/782 [==============================] - 7s 9ms/step - loss: 0.6104 - accuracy: 0.8431
